In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
#Data Ingestion
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/observability/tutorials/observability")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='\n\n\n\n\nAdd observability to your LLM application | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesUse monitoring chartsDashboardsLog traces to specific projectSet up automation rulesOnline EvaluationSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog mu

In [8]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Add observability to your LLM application | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilt

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [12]:
print(vectorstoredb)

In [13]:
## Query From a vector db
query="Having observability set up from the start can you help iterate much more quickly "
result=vectorstoredb.similarity_search(query)
result[0].page_content

"Prototyping\u200b\nHaving observability set up from the start can you help iterate much more quickly than you would otherwise be able to.\nIt allows you to have great visibility into your application as you are rapidly iterating on the prompt, or changing the data and models you are using.\nIn this section we'll walk through how to set up observability so you can have maximal observability as you are prototyping.\nSet up your environment\u200b\nFirst, create an API key by navigating to the settings page.\nNext, install the LangSmith SDK:\nPython SDKTypeScript SDKpip install langsmithnpm install langsmith\nFinally, set up the appropriate environment variables. This will log traces to the default project (though you can easily change that).\nexport LANGSMITH_TRACING=trueexport LANGSMITH_API_KEY=<your-api-key>export LANGSMITH_PROJECT=default"

In [15]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [16]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001FBD98C43D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FBD98CB8B0>, root_client=<openai.OpenAI object at 0x000001FBD16D5450>, root_async_client=<openai.AsyncOpenAI object at 0x000001FBD98C53C0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [17]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Having observability set up from the start can you help iterate much more quickly ",
    "context":[Document(page_content="Having observability set up from the start can you help iterate much more quickly than you would otherwise be able to. It allows you to have great visibility into your application as you are rapidly iterating on the prompt, or changing the data and models you are using. In this section we'll walk through how to set up observability so you can have maximal observability as you are prototyping. ")]
})

'The provided context emphasizes that having observability set up from the start helps you iterate much more quickly than you otherwise would be able to. It provides great visibility into your application as you rapidly iterate on the prompt, or change the data and models you are using. The context also mentions that there will be a walkthrough on how to set up observability to achieve maximal observability while prototyping.'

In [18]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [19]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [20]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FBD9023F10>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [21]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Having observability set up from the start can you help iterate much more quickly "})
response['answer']

'What is the importance of observability in LLM applications?  \n\nObservability is crucial for LLM applications because LLMs are non-deterministic by nature and can produce unexpected results, making them more challenging to debug than typical software applications. Observability provides you with great visibility and meaningful insights into your application throughout all stages of development, from prototyping to production. It helps in quickly iterating during development and continuously monitoring performance once the application is live, allowing you to address issues efficiently and ensure the application performs well.'

In [22]:
response

{'input': 'Having observability set up from the start can you help iterate much more quickly ',
 'context': [Document(id='59cd853d-14fc-4657-b95d-1d38e5f4ce9a', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="Prototyping\u200b\nHaving observability set up from the start can you help iterate much more quickly than you would otherwise be able to.\nIt allows you to have great visibility into your application as you are rapidly iterating on the prompt, or changing the data and models you are using.\nIn this section we'll walk through how to set up observability so you can have maximal observability as you are prototyping.\nSet up your environment\u200b\nFirst, create an API key by navigating to the settings page.\nNext, install t

In [23]:
response["context"]

[Document(id='59cd853d-14fc-4657-b95d-1d38e5f4ce9a', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="Prototyping\u200b\nHaving observability set up from the start can you help iterate much more quickly than you would otherwise be able to.\nIt allows you to have great visibility into your application as you are rapidly iterating on the prompt, or changing the data and models you are using.\nIn this section we'll walk through how to set up observability so you can have maximal observability as you are prototyping.\nSet up your environment\u200b\nFirst, create an API key by navigating to the settings page.\nNext, install the LangSmith SDK:\nPython SDKTypeScript SDKpip install langsmithnpm install langsmith\nFinally, set up the a